In [1]:
import geopandas as gpd
from src.organized_datasets_creation.utils import resolve_nominatim_city_name
from src.graph_layering.create_dataframes import create_osmnx_dataframes
from src.organized_datasets_creation.utils import convert_nominatim_name_to_filename
from src.graph_layering.graph_layer_creator import GraphLayerController
import pandas as pd
from typing import cast
import os
from src.graph_layering.graph_layer_creator import SourceType
import warnings
from src.graph_layering.create_hetero_data import create_hetero_data

from tqdm import tqdm

from tqdm import tqdm
import wandb.util
import wandb
import os

import glob
import numpy as np

In [ ]:
WANDB_API_KEY = os.environ.get("WANDB_API_KEY", None)
assert (
    WANDB_API_KEY is not None
), "WANDB_API_KEY is not set, did you forget it in the config file?"

In [2]:
# general settings
ORGANIZED_DATASETS_LOCATION = (
    "/home/staszek/mgr/gradient/gradient/data/organized-datasets"
)

# downstream task settings
ACCIDENTS_LOCATION = "/home/staszek/mgr/gradient/gradient/data/wypadki-pl/accidents.csv"
TRAIN_SAVE_DIR = "/media/staszek/m2-mint/gradient_logs/"

SWEEP_RUNS_COUNT = 50
EPOCHS = 300

H3_RESOLUTION = 9
H3_YEAR = 2017
H3_EMBEDDING_METHOD = "count-embedder"

DATASET_CONTRUCTION_METHOD = "drop-non-matching-columns"

WANDB_SWEEP_PARAMS = {
    "method": "bayes",
    "metric": {"name": "mean_f1", "goal": "maximize"},
    "parameters": {
        "hidden_channels": {"values": [10, 20, 30, 40, 50]},
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
        "num_conv_layers": {"values": [1, 2, 3, 4, 5]},
        "lin_layer_size": {"values": [8, 16, 32, 64, 128]},
        "num_lin_layers": {"values": [0, 1, 2, 3, 4]},
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2,
        },
    },
}

In [3]:
available_h3_embedding_methods = glob.glob(ORGANIZED_DATASETS_LOCATION + "/**/**/**/*")
available_h3_embedding_methods = [
    os.path.basename(x) for x in available_h3_embedding_methods
]
available_h3_embedding_methods = np.unique(available_h3_embedding_methods).tolist()
print("Available h3 embedding methods:")
assert H3_EMBEDDING_METHOD in available_h3_embedding_methods, (
    f"Chosen h3 embedding method is not available. "
    f"Available methods are: {available_h3_embedding_methods}"
)
print(available_h3_embedding_methods)

['count-embedder', 'hex2vec', 'highway2vec']

In [4]:
accidents = gpd.read_file(ACCIDENTS_LOCATION)

In [5]:
cities = list(map(lambda x: x + ", Poland", accidents["mie_nazwa"].unique()))
print("Cities:")
print(cities)

['Wrocław, Poland',
 'Szczecin, Poland',
 'Poznań, Poland',
 'Kraków, Poland',
 'Warszawa, Poland']

In [7]:
def create_gdfs(city_name: str, h3_resolution: int, year: int, method: str):
    osmnx_nodes, osmnx_edges = create_osmnx_dataframes(
        df_accidents=accidents, nominatim_city_name=city_name
    )
    assert (
        method in available_h3_embedding_methods
    ), f"H3 embedding method {method} not available, available methods: {available_h3_embedding_methods}"
    hexes: gpd.GeoDataFrame = gpd.read_parquet(
        os.path.join(
            ORGANIZED_DATASETS_LOCATION,
            f"{convert_nominatim_name_to_filename(resolve_nominatim_city_name(city_name))}/{year}/h{h3_resolution}/{method}/dataset.parquet",
        )
    )
    hexes = (
        hexes.rename(columns={"region_id": "h3_id"})
        .rename_axis("region_id", axis=0)
        .drop(columns="accidents_count")
    )  # we will be using different aggregation type than the one in the dataset

    return dict(osmnx_nodes=osmnx_nodes, osmnx_edges=osmnx_edges, hexes=hexes)


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print("Creating gdfs...")
    gdfs_dict = {
        city_name: create_gdfs(
            city_name, H3_RESOLUTION, H3_YEAR, method=H3_EMBEDDING_METHOD
        )
        for city_name in tqdm(cities)
    }

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:25<01:42, 25.74s/it]

KeyboardInterrupt: 

# Usuwanie kolumn, które nie są wspólne dla wszystkich miast


In [ ]:
def get_presence_df(gdfs_dict, tested_df_name):
    presence_df = pd.DataFrame(
        list(
            map(
                lambda v: (v[0], v[1][tested_df_name].columns.to_list()),
                list(gdfs_dict.items()),
            )
        ),
        columns=["city_name", "col"],
    ).explode("col")
    presence_df = (
        pd.get_dummies(presence_df, columns=["col"], prefix="", prefix_sep="")
        .groupby("city_name")
        .sum()
    )
    return presence_df


def filter_presence_df(df):
    return df.loc[:, df.sum(axis=0) == len(cities)]


def get_common_columns(gdfs_dict, tested_df_name):
    df_columns_presence = get_presence_df(gdfs_dict, tested_df_name)
    df_common_columns = filter_presence_df(df_columns_presence)
    return df_common_columns

In [ ]:
df_osmnx_node_common_columns = get_common_columns(gdfs_dict, "osmnx_nodes")
df_osmnx_node_common_columns

,accidents_count,crossing,geometry,mini_roundabout,motorway_junction,osmid,street_count,traffic_signals,turning_circle,x,y
city_name,,,,,,,,,,,
"Kraków, Poland",1,1,1,1,1,1,1,1,1,1,1
"Poznań, Poland",1,1,1,1,1,1,1,1,1,1,1
"Szczecin, Poland",1,1,1,1,1,1,1,1,1,1,1
"Warsaw, Poland",1,1,1,1,1,1,1,1,1,1,1
"Wrocław, Poland",1,1,1,1,1,1,1,1,1,1,1


In [ ]:
df_osmnx_edge_common_columns = get_common_columns(gdfs_dict, "osmnx_edges")
df_osmnx_edge_common_columns

,access_0,access_destination,access_no,access_permissive,access_yes,bridge_0,bridge_viaduct,bridge_yes,geometry,highway_living_street,...,length,maxspeed,oneway,reversed,tunnel_0,tunnel_building_passage,tunnel_yes,u,v,width
city_name,,,,,,,,,,,,,,,,,,,,,
"Kraków, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Poznań, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Szczecin, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Warsaw, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Wrocław, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
df_hexes_common_columns = get_common_columns(gdfs_dict, "hexes")
df_hexes_common_columns

,aeroway_aerodrome,aeroway_helipad,aeroway_runway,amenity_animal_shelter,amenity_arts_centre,amenity_atm,amenity_bank,amenity_bar,amenity_bbq,amenity_bench,...,water_pond,water_reservoir,water_river,water_wastewater,waterway_canal,waterway_ditch,waterway_drain,waterway_river,waterway_stream,waterway_weir
city_name,,,,,,,,,,,,,,,,,,,,,
"Kraków, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Poznań, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Szczecin, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Warsaw, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Wrocław, Poland",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
print("Deleting columns that are not common for all cities...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdf_for_city in gdfs_dict.values():
        osmnx_nodes = gdf_for_city["osmnx_nodes"]
        osmnx_edges = gdf_for_city["osmnx_edges"]
        hexes = gdf_for_city["hexes"]

        osmnx_nodes.drop(
            columns=osmnx_nodes.columns.difference(
                df_osmnx_node_common_columns.columns
            ),
            inplace=True,
        )
        osmnx_edges.drop(
            columns=osmnx_edges.columns.difference(
                df_osmnx_edge_common_columns.columns
            ),
            inplace=True,
        )
        hexes.drop(
            columns=hexes.columns.difference(df_hexes_common_columns.columns),
            inplace=True,
        )

        gdf_for_city["osmnx_nodes"] = osmnx_nodes.reindex(
            columns=df_osmnx_node_common_columns.columns
        )
        gdf_for_city["osmnx_edges"] = osmnx_edges.reindex(
            columns=df_osmnx_edge_common_columns.columns
        )
        gdf_for_city["hexes"] = hexes.reindex(columns=df_hexes_common_columns.columns)

        gdf_for_city["controller"] = GraphLayerController(
            gdf_for_city["hexes"],
            gdf_for_city["osmnx_nodes"],
            gdf_for_city["osmnx_edges"],
        )

/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradien

In [ ]:
def patch_hexes_with_y(
    osmnx_nodes: gpd.GeoDataFrame,
    hexes: gpd.GeoDataFrame,
    controller: GraphLayerController,
):
    virtual_edges = controller.get_virtual_edges_to_hexes(SourceType.OSMNX_NODES)
    hexes_with_y = cast(
        gpd.GeoDataFrame,
        hexes.merge(
            virtual_edges.merge(osmnx_nodes, left_on="source_id", right_index=True)[
                ["region_id", "accidents_count"]
            ]
            .groupby("region_id")
            .sum(),
            left_index=True,
            right_index=True,
            how="left",
        ).fillna(0),
    )
    hexes_with_y["accident_occured"] = (hexes_with_y["accidents_count"] > 0).astype(int)
    hexes_with_y.drop(columns="accidents_count", inplace=True)
    controller.hexes_gdf = hexes_with_y
    controller._hexes_centroids_gdf = controller._create_hexes_centroids_gdf()

In [ ]:
print("Patching hexes with y...")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for gdfs in gdfs_dict.values():
        patch_hexes_with_y(gdfs["osmnx_nodes"], gdfs["hexes"], gdfs["controller"])

/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradient/gradient/src/graph_layering/graph_layer_creator.py:410: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.hexes_gdf.centroid, columns=["centroid_geometry"]
/home/staszek/mgr/gradien

In [ ]:
def create_torch_geometric_hetero_data(
    osmnx_nodes, osmnx_edges, hexes, controller: GraphLayerController
):
    edges_attr_columns = osmnx_edges.columns[
        ~osmnx_edges.columns.isin(["u", "v", "key", "geometry"])
    ]
    nodes_attr_columns = osmnx_nodes.columns[
        ~osmnx_nodes.columns.isin(["geometry", "x", "y", "osmid", "accidents_count"])
    ]
    hexes_attr_columns = hexes.columns[~hexes.columns.isin(["geometry", "h3_id"])]

    data = create_hetero_data(
        controller,
        hexes_attrs_columns_names=hexes_attr_columns,
        osmnx_edge_attrs_columns_names=edges_attr_columns,
        osmnx_node_attrs_columns_names=nodes_attr_columns,
        virtual_edge_attrs_columns_names=[],
        hexes_y_columns_names=["accident_occured"],
    )
    return data


data_dict = {
    city_name: create_torch_geometric_hetero_data(**gdfs)
    for city_name, gdfs in gdfs_dict.items()
}

In [ ]:
def shift_elements_right(lst):
    shifted_lst = [lst[-1]] + lst[:-1]
    return shifted_lst


cities_names_list = list(data_dict.keys())

# val + test
folds_tuples = list(zip(shift_elements_right(cities_names_list), cities_names_list))

In [ ]:
print("Folds tuples (val, test)")
print(folds_tuples)

[('Kraków, Poland', 'Wrocław, Poland'),
 ('Wrocław, Poland', 'Warsaw, Poland'),
 ('Warsaw, Poland', 'Szczecin, Poland'),
 ('Szczecin, Poland', 'Poznań, Poland'),
 ('Poznań, Poland', 'Kraków, Poland')]

In [ ]:
from wandb.util import generate_id

from src.training.train import train


def run_k_fold():
    run = wandb.init()
    epochs = EPOCHS

    config = wandb.config

    if hasattr(config, "lin_layer_size") and hasattr(config, "num_lin_layers"):
        lin_layer_sizes = [config.lin_layer_size] * config.num_lin_layers
    else:
        lin_layer_sizes = config.lin_layer_sizes

    hparams = {
        "hidden_channels": config.hidden_channels,
        "lr": config.learning_rate,
        "num_conv_layers": config.num_conv_layers,
        "lin_layer_sizes": lin_layer_sizes,
        "weight_decay": config.weight_decay,
    }

    aucs = []
    accuracies = []
    f1s = []

    fold_group_id = generate_id()
    for index, (val_city_name, test_city_name) in enumerate(folds_tuples):
        val_data = [data_dict[val_city_name].to("cpu").clone()]
        train_data = [
            v.to("cpu").clone()
            for k, v in data_dict.items()
            if k != val_city_name and k != test_city_name
        ]
        test_data = data_dict[test_city_name].to("cpu").clone()

        auc, accuracy, f1, model_path = train(
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            epochs=epochs,
            hparams=hparams,
            train_save_dir=TRAIN_SAVE_DIR,
        )
        run.log_model(
            path=model_path,
            name=f"model_{fold_group_id}_fold_{index}",
        )
        run.log({f"auc_fold_{index}": auc})
        run.log({f"accuracy_fold_{index}": accuracy})
        run.log({f"f1_fold_{index}": f1})

        aucs.append(auc)
        accuracies.append(accuracy)
        f1s.append(f1)

    mean_auc = sum(aucs) / len(aucs)
    mean_accuracy = sum(accuracies) / len(accuracies)
    mean_f1 = sum(f1s) / len(f1s)
    run.log({"mean_auc": mean_auc})
    run.log({"mean_accuracy": mean_accuracy})
    run.log({"mean_f1": mean_f1})


def main():
    try:
        wandb.login(key=WANDB_API_KEY)
        sweep_id = wandb.sweep(WANDB_SWEEP_PARAMS, project="accidents-downstream-task")
        wandb.agent(sweep_id, function=run_k_fold, count=SWEEP_RUNS_COUNT)
    except Exception as e:
        print(e)
        wandb.finish()
        raise e


main()